In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys, cv2
import torch
import torchvision
import torch.nn as nn
import numpy as np
import os, time, cv2, sys
from PIL import Image
import torchvision.transforms as transforms


sz=224
num_classes=4
proj_dir = '/kaggle/input/cassava-leaf-disease-classification/'




In [ ]:
def softmax(X, theta=1.0, axis=None):
    y = np.atleast_2d(X)

    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    y = y * float(theta)
    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis=axis), axis)
    # exponentiate y
    y = np.exp(y)
    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis=axis), axis)
    p = y / ax_sum
    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()
    return p

In [ ]:
def light_model(num_classes):
    squeezenet_custom=torchvision.models.squeezenet1_0(pretrained=False)

    classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Conv2d(in_channels=512
                  , out_channels=num_classes
                  , kernel_size=(1, 1)
                  , stride=(1, 1)
                  , padding=(1, 1)),
        nn.ReLU(inplace=True),
        nn.AdaptiveAvgPool2d((1, 1))
    )

    squeezenet_custom.classifier = classifier
    return squeezenet_custom


def vgg_16(num_classes):
        model = torchvision.models.vgg16(pretrained=False)

        classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
            )

        model.classifier = classifier
        return model

squeezenet_custom_4 = light_model(4)
squeezenet_custom_2 = light_model(2)
vgg_custom_2=vgg_16(2)

In [ ]:
# Data Transforms

leaf_transform=transforms.Compose([
    transforms.CenterCrop(400)
    , transforms.Resize(size=(224, 224))
    
    , transforms.ToTensor()
    , transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
minority_idx = {
    0: 0
    , 1: 1
    , 2: 2
    , 3: 4    
}

binary_idx = {
    0: '0_1_2_4'
    , 1: 3
}

In [ ]:
weight_dir='../input/cassava-models-2'
print(os.listdir(weight_dir))
model_1_path=os.path.join('../input/cassava-models', 'minority_weights.pth')
model_2_path=os.path.join(weight_dir, 'binary_weights_2.pth')

In [ ]:
model_1=torch.load(model_1_path, map_location=torch.device('cuda'))
weights_1 = model_1.state_dict()
squeezenet_custom_4.load_state_dict(weights_1)
squeezenet_custom_4.eval()

model_2=torch.load(model_2_path, map_location=torch.device('cuda'))
weights_2 = model_2.state_dict()
vgg_custom_2.load_state_dict(weights_2)
vgg_custom_2.eval()

In [ ]:
def prediction_logic(img, squeezenet_custom_4, squeezenet_custom_2, thresh_3=0.50):
    
    preds_minority = softmax(squeezenet_custom_4(img).cpu().detach().numpy())
    preds_binary = softmax(squeezenet_custom_2(img).cpu().detach().numpy())
    binary_cls=np.argmax(preds_binary)
    minority_cls=np.argmax(preds_minority)
    
    cls_3 = preds_binary[0][1]
    print(preds_minority, preds_binary)
    if cls_3 <= thresh_3:
        return minority_idx[minority_cls]
    else: 
        return binary_idx[binary_cls]

In [ ]:
def img_transform(img_path):
    img = Image.open(img_path).convert('RGB')
    r,g,b = img.split()
    img = Image.merge("RGB", (b,g,r))

    img = leaf_transform(img).float()
    img = img.unsqueeze(0)
    return img

In [ ]:
train_dir=(os.path.join(proj_dir, 'train_images'))
test_dir=(os.path.join(proj_dir, 'test_images'))

df = pd.read_csv(os.path.join(proj_dir, 'train.csv'))
sample_df = pd.read_csv(os.path.join(proj_dir, 'sample_submission.csv'))
sample_df.head()

In [ ]:

test_preds=[]
for i, j in enumerate(os.listdir(test_dir)):
    print(i, j)
    img_path=os.path.join(test_dir, j)
    img=img_transform(img_path)
    final_pred=prediction_logic(img, squeezenet_custom_4, squeezenet_custom_2)
    test_preds.append([j,final_pred])
print(test_preds)

In [ ]:
sub = pd.DataFrame.from_records(test_preds, columns=['image_id', 'label'])
sub.head()

In [ ]:
os.chdir('/kaggle/working/')
sub.to_csv("/kaggle/working/submission.csv", index=False)